In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import dash_table
# Import plotly
import plotly.graph_objs as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [16]:
%load_ext watermark

%watermark -v -m -p pandas,numpy,sklearn,plotly,dash,watermark

print(" ")
%watermark -u -n -t -z

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

pandas   : 1.2.4
numpy    : 1.19.5
sklearn  : 0.24.1
plotly   : 5.1.0
dash     : 1.20.0
watermark: 2.2.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit

 
Last updated: Mon Jul 05 2021 11:42:04Eastern Daylight Time



In [17]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import os
import multiprocessing as mp
import io
import psycopg2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import unicodedata

In [18]:
from tensorflow import keras

In [19]:
# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [20]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation, concatenate, Dense, Dropout, Embedding, Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [21]:
def sqlDF(cur, sql):
    ''' create pandas df from sql query'''
    cur.execute(sql)
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    df =  pd.DataFrame(data)
    df.columns = colnames
    return df
def sqlConnect():
    ''' connect to postgres server'''
    conn = psycopg2.connect(host = "baseball.cdvpibivs0o7.us-east-2.rds.amazonaws.com",
                            port = 5432,
                            database = "baseball",
                            user = "postgres",
                            password = "77DDFPlfH9FIjylZv8IX",
                            connect_timeout=3)
    return conn.cursor()

In [75]:
%%time
sql = '''SELECT release_speed, stand, p_throws, type, balls, strikes, pfx_x, pfx_z, inning,
game_type, plate_x, plate_z, daynight, outs_when_up, home_score, bat_score, fld_score,
umpname, height_in FROM pitches_expanded;'''
pitch = sqlDF(sqlConnect(), sql)
pitch

Wall time: 40.1 s


,release_speed,stand,p_throws,type,balls,strikes,pfx_x,pfx_z,inning,game_type,plate_x,plate_z,daynight,outs_when_up,home_score,bat_score,fld_score,umpname,height_in
0,91.5,L,R,S,0.0,0.0,-0.604558,1.386800,7.0,R,-0.577,2.692,day,0.0,0.0,0.0,3.0,Bill Miller,76
1,94.6,L,L,B,1.0,0.0,0.355692,1.922867,6.0,R,-0.721,1.500,day,2.0,0.0,4.0,0.0,Jim Joyce,72
2,95.2,R,R,B,0.0,1.0,-0.472350,2.027500,6.0,R,1.215,2.066,day,1.0,3.0,5.0,3.0,Angel Campos,78
3,94.8,R,R,S,0.0,0.0,-0.419467,1.855500,6.0,R,0.282,1.922,day,1.0,3.0,5.0,3.0,Angel Campos,78
4,75.3,R,L,S,0.0,0.0,-0.337358,0.159867,6.0,R,-0.544,2.035,night,0.0,2.0,4.0,2.0,Dan Bellino,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460633,93.9,R,R,B,1.0,0.0,-1.374150,0.615667,6.0,R,0.556,1.429,day,1.0,1.0,1.0,1.0,Vic Carapazza,72
1460634,85.9,L,L,B,2.0,0.0,-0.502967,0.321833,6.0,R,-0.840,3.494,day,2.0,0.0,4.0,0.0,Jim Joyce,72
1460635,94.9,R,R,B,0.0,0.0,-1.592642,0.814900,6.0,R,-0.725,1.605,day,1.0,1.0,1.0,1.0,Vic Carapazza,72
1460636,88.6,R,L,S,0.0,0.0,0.750925,1.209067,5.0,R,-0.342,1.758,day,1.0,0.0,0.0,4.0,Paul Emmel,72


In [76]:
pitch.to_csv('pitch.csv')
pitch = pd.read_csv('pitch.csv', index_col=0)

In [77]:
pitch

,release_speed,stand,p_throws,type,balls,strikes,pfx_x,pfx_z,inning,game_type,plate_x,plate_z,daynight,outs_when_up,home_score,bat_score,fld_score,umpname,height_in
0,91.5,L,R,S,0.0,0.0,-0.604558,1.386800,7.0,R,-0.577,2.692,day,0.0,0.0,0.0,3.0,Bill Miller,76
1,94.6,L,L,B,1.0,0.0,0.355692,1.922867,6.0,R,-0.721,1.500,day,2.0,0.0,4.0,0.0,Jim Joyce,72
2,95.2,R,R,B,0.0,1.0,-0.472350,2.027500,6.0,R,1.215,2.066,day,1.0,3.0,5.0,3.0,Angel Campos,78
3,94.8,R,R,S,0.0,0.0,-0.419467,1.855500,6.0,R,0.282,1.922,day,1.0,3.0,5.0,3.0,Angel Campos,78
4,75.3,R,L,S,0.0,0.0,-0.337358,0.159867,6.0,R,-0.544,2.035,night,0.0,2.0,4.0,2.0,Dan Bellino,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460633,93.9,R,R,B,1.0,0.0,-1.374150,0.615667,6.0,R,0.556,1.429,day,1.0,1.0,1.0,1.0,Vic Carapazza,72
1460634,85.9,L,L,B,2.0,0.0,-0.502967,0.321833,6.0,R,-0.840,3.494,day,2.0,0.0,4.0,0.0,Jim Joyce,72
1460635,94.9,R,R,B,0.0,0.0,-1.592642,0.814900,6.0,R,-0.725,1.605,day,1.0,1.0,1.0,1.0,Vic Carapazza,72
1460636,88.6,R,L,S,0.0,0.0,0.750925,1.209067,5.0,R,-0.342,1.758,day,1.0,0.0,0.0,4.0,Paul Emmel,72


In [78]:
pitch['height_in'].describe()

count    1.460638e+06
mean     7.475215e+01
std      2.181669e+00
min      6.700000e+01
25%      7.300000e+01
50%      7.500000e+01
75%      7.600000e+01
max      8.300000e+01
Name: height_in, dtype: float64

In [80]:
pitch['type'].unique()

array(['S', 'B'], dtype=object)

In [81]:
pitch['type'].value_counts()

S    740622
B    720016
Name: type, dtype: int64

In [82]:
len(pitch['umpname'].unique())

129

In [83]:
len(pitch)

1460638

In [84]:
cutoffs = {}
percentile_cutoff = 0.99

counts_list = list(pitch['umpname'].value_counts())
player_counts = list(pitch['umpname'].value_counts().index)
total_at_bats = sum(counts_list)

cumulative_percentage = [sum(counts_list[:i + 1]) / total_at_bats for i in range(len(counts_list))]
cutoff_index = sum([1 for total in cumulative_percentage if total <= percentile_cutoff])

cutoff = player_counts[:cutoff_index]
cutoffs['umpire'] = cutoff
print("Original: {0}\tNew: {1}\tProportion: {2:.2f}".format(
        len(pitch['umpname'].unique()), cutoff_index, cutoff_index / len(pitch['umpname'].unique())))

Original: 129	New: 108	Proportion: 0.84


In [85]:
pitch = pitch[pitch['umpname'].isin(cutoffs['umpire'])]

In [86]:
pitch['umpname'].value_counts()

James Hoye         22129
Marvin Hudson      20549
Tim Timmons        20301
Angel Hernandez    20284
Tom Hallion        19800
                   ...  
Jeremie Rehak       2728
Ryan Additon        2577
Bill Hohn           2534
Nick Mahrley        2505
Jerry Crawford      2481
Name: umpname, Length: 108, dtype: int64

In [87]:
pitch = pitch[pitch['game_type']=='R']
pitch['release_speed'] = (pitch['release_speed']-pitch['release_speed'].min())/(pitch['release_speed'].max()-pitch['release_speed'].min())
pitch['stand'] = np.where(pitch['stand']=='R', 1, 0)
pitch['p_throws'] = np.where(pitch['p_throws']=='R', 1, 0)
pitch['type'] = np.where(pitch['type']=='B', 1, 0)
pitch['day'] = np.where(pitch['daynight']=='day', 1, 0)
pitch['b_home'] = np.where(pitch['bat_score']==pitch['home_score'],1,0)
pitch['height_in'] = (pitch['height_in']-pitch['height_in'].min())/(pitch['height_in'].max()-pitch['height_in'].min())
pitch.reset_index(drop=True, inplace=True)
pitch

,release_speed,stand,p_throws,type,balls,strikes,pfx_x,pfx_z,inning,game_type,...,plate_z,daynight,outs_when_up,home_score,bat_score,fld_score,umpname,height_in,day,b_home
0,0.789902,0,1,0,0.0,0.0,-0.604558,1.386800,7.0,R,...,2.692,day,0.0,0.0,0.0,3.0,Bill Miller,0.5625,1,1
1,0.840391,0,0,1,1.0,0.0,0.355692,1.922867,6.0,R,...,1.500,day,2.0,0.0,4.0,0.0,Jim Joyce,0.3125,1,0
2,0.850163,1,1,1,0.0,1.0,-0.472350,2.027500,6.0,R,...,2.066,day,1.0,3.0,5.0,3.0,Angel Campos,0.6875,1,0
3,0.843648,1,1,0,0.0,0.0,-0.419467,1.855500,6.0,R,...,1.922,day,1.0,3.0,5.0,3.0,Angel Campos,0.6875,1,0
4,0.526059,1,0,0,0.0,0.0,-0.337358,0.159867,6.0,R,...,2.035,night,0.0,2.0,4.0,2.0,Dan Bellino,0.4375,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429159,0.828990,1,1,1,1.0,0.0,-1.374150,0.615667,6.0,R,...,1.429,day,1.0,1.0,1.0,1.0,Vic Carapazza,0.3125,1,1
1429160,0.698697,0,0,1,2.0,0.0,-0.502967,0.321833,6.0,R,...,3.494,day,2.0,0.0,4.0,0.0,Jim Joyce,0.3125,1,0
1429161,0.845277,1,1,1,0.0,0.0,-1.592642,0.814900,6.0,R,...,1.605,day,1.0,1.0,1.0,1.0,Vic Carapazza,0.3125,1,1
1429162,0.742671,1,0,0,0.0,0.0,0.750925,1.209067,5.0,R,...,1.758,day,1.0,0.0,0.0,4.0,Paul Emmel,0.3125,1,1


In [88]:
pitch.columns

Index(['release_speed', 'stand', 'p_throws', 'type', 'balls', 'strikes',
       'pfx_x', 'pfx_z', 'inning', 'game_type', 'plate_x', 'plate_z',
       'daynight', 'outs_when_up', 'home_score', 'bat_score', 'fld_score',
       'umpname', 'height_in', 'day', 'b_home'],
      dtype='object')

In [90]:
features = ['stand', 'p_throws', 'type', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'inning', 'plate_x', 'plate_z', 'day',
            'outs_when_up', 'bat_score', 'fld_score', 'b_home', 'height_in', 'day', 'umpname']
data_final = pitch[features]
data_final

,stand,p_throws,type,balls,strikes,pfx_x,pfx_z,inning,plate_x,plate_z,day,outs_when_up,bat_score,fld_score,b_home,height_in,day,umpname
0,0,1,0,0.0,0.0,-0.604558,1.386800,7.0,-0.577,2.692,1,0.0,0.0,3.0,1,0.5625,1,Bill Miller
1,0,0,1,1.0,0.0,0.355692,1.922867,6.0,-0.721,1.500,1,2.0,4.0,0.0,0,0.3125,1,Jim Joyce
2,1,1,1,0.0,1.0,-0.472350,2.027500,6.0,1.215,2.066,1,1.0,5.0,3.0,0,0.6875,1,Angel Campos
3,1,1,0,0.0,0.0,-0.419467,1.855500,6.0,0.282,1.922,1,1.0,5.0,3.0,0,0.6875,1,Angel Campos
4,1,0,0,0.0,0.0,-0.337358,0.159867,6.0,-0.544,2.035,0,0.0,4.0,2.0,0,0.4375,0,Dan Bellino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429159,1,1,1,1.0,0.0,-1.374150,0.615667,6.0,0.556,1.429,1,1.0,1.0,1.0,1,0.3125,1,Vic Carapazza
1429160,0,0,1,2.0,0.0,-0.502967,0.321833,6.0,-0.840,3.494,1,2.0,4.0,0.0,0,0.3125,1,Jim Joyce
1429161,1,1,1,0.0,0.0,-1.592642,0.814900,6.0,-0.725,1.605,1,1.0,1.0,1.0,1,0.3125,1,Vic Carapazza
1429162,1,0,0,0.0,0.0,0.750925,1.209067,5.0,-0.342,1.758,1,1.0,0.0,4.0,1,0.3125,1,Paul Emmel


In [91]:
data = {}
data['game_condition'] = np.array(data_final[['stand', 'p_throws', 'type', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'inning', 
                                              'plate_x', 'plate_z', 'day', 'outs_when_up', 'bat_score', 'fld_score', 'b_home',
                                              'height_in', 'day']])
data['game_condition']

array([[0.    , 1.    , 0.    , ..., 0.5625, 1.    , 1.    ],
       [0.    , 0.    , 1.    , ..., 0.3125, 1.    , 1.    ],
       [1.    , 1.    , 1.    , ..., 0.6875, 1.    , 1.    ],
       ...,
       [1.    , 1.    , 1.    , ..., 0.3125, 1.    , 1.    ],
       [1.    , 0.    , 0.    , ..., 0.3125, 1.    , 1.    ],
       [0.    , 1.    , 1.    , ..., 0.3125, 1.    , 1.    ]])

In [103]:
data['game_condition'].shape

(1429164, 19)

In [93]:
ump_unique = data_final['umpname'].unique()
cat_to_int = {ump_unique[i]: i for i in range(len(ump_unique))}
cat_to_int

{'Bill Miller': 0,
 'Jim Joyce': 1,
 'Angel Campos': 2,
 'Dan Bellino': 3,
 'Paul Emmel': 4,
 'Doug Eddings': 5,
 'Vic Carapazza': 6,
 'Adrian Johnson': 7,
 'Larry Vanover': 8,
 'Marty Foster': 9,
 'Kerwin Danley': 10,
 'Lance Barrett': 11,
 'James Hoye': 12,
 'Hunter Wendelstedt': 13,
 'Manny Gonzalez': 14,
 'Tim McClelland': 15,
 'Fieldin Culbreth': 16,
 'Eric Cooper': 17,
 'Dan Iassogna': 18,
 'Gary Darling': 19,
 'Greg Gibson': 20,
 'Bob Davidson': 21,
 'Marvin Hudson': 22,
 'Todd Tichenor': 23,
 'Jeff Kellogg': 24,
 'Jim Wolf': 25,
 'Dana DeMuth': 26,
 'Mike DiMuro': 27,
 'CB Bucknor': 28,
 'Paul Nauert': 29,
 'Tony Randazzo': 30,
 'Jim Reynolds': 31,
 'Tim Welke': 32,
 'Mike Everitt': 33,
 'Jerry Meals': 34,
 'Scott Barry': 35,
 'Bill Welke': 36,
 'Phil Cuzzi': 37,
 'Tim Timmons': 38,
 'Ed Hickox': 39,
 'Ron Kulpa': 40,
 'Sam Holbrook': 41,
 'Laz Diaz': 42,
 'Gerry Davis': 43,
 'Mike Winters': 44,
 'Brian Knight': 45,
 'Alan Porter': 46,
 'Dale Scott': 47,
 'Mark Carlson': 48,
 '

In [95]:
cat_to_int_rev = {value:key for (key, value) in cat_to_int.items()}
cat_to_int_rev

{0: 'Bill Miller',
 1: 'Jim Joyce',
 2: 'Angel Campos',
 3: 'Dan Bellino',
 4: 'Paul Emmel',
 5: 'Doug Eddings',
 6: 'Vic Carapazza',
 7: 'Adrian Johnson',
 8: 'Larry Vanover',
 9: 'Marty Foster',
 10: 'Kerwin Danley',
 11: 'Lance Barrett',
 12: 'James Hoye',
 13: 'Hunter Wendelstedt',
 14: 'Manny Gonzalez',
 15: 'Tim McClelland',
 16: 'Fieldin Culbreth',
 17: 'Eric Cooper',
 18: 'Dan Iassogna',
 19: 'Gary Darling',
 20: 'Greg Gibson',
 21: 'Bob Davidson',
 22: 'Marvin Hudson',
 23: 'Todd Tichenor',
 24: 'Jeff Kellogg',
 25: 'Jim Wolf',
 26: 'Dana DeMuth',
 27: 'Mike DiMuro',
 28: 'CB Bucknor',
 29: 'Paul Nauert',
 30: 'Tony Randazzo',
 31: 'Jim Reynolds',
 32: 'Tim Welke',
 33: 'Mike Everitt',
 34: 'Jerry Meals',
 35: 'Scott Barry',
 36: 'Bill Welke',
 37: 'Phil Cuzzi',
 38: 'Tim Timmons',
 39: 'Ed Hickox',
 40: 'Ron Kulpa',
 41: 'Sam Holbrook',
 42: 'Laz Diaz',
 43: 'Gerry Davis',
 44: 'Mike Winters',
 45: 'Brian Knight',
 46: 'Alan Porter',
 47: 'Dale Scott',
 48: 'Mark Carlson',
 4

In [96]:
data['type'] = np.array(data_final['type'])
data['type'].shape

(1429164,)

In [99]:
data['umpname'] = np.array(data_final['umpname'].replace(cat_to_int))
data['umpname'] = data['umpname'].reshape((len(data['umpname']), 1))
data['umpname'][12111]

array([7], dtype=int64)

In [101]:
len(data_final['umpname'].unique())

108

In [107]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation, concatenate, Dense, Dropout, Embedding, Input, Reshape
from tensorflow.keras.models import Model

NUM_UMPIRES = len(data_final['umpname'].unique())
VEC_SIZE = 16
ACTIVATION = "sigmoid"
GAME_VEC_INPUT = data['game_condition'].shape[1]
GAME_VEC_SIZE = 6


umpire_idx = Input(shape = (1, ), dtype = "int32", name = "umpire_idx")
umpire_embed = Embedding(NUM_UMPIRES, VEC_SIZE, input_length = 1)(umpire_idx)
umpire_embed = Reshape((VEC_SIZE, ), name = "umpire_embed")(umpire_embed)
umpire_embed = Activation(ACTIVATION)(umpire_embed)

game_idx = Input(shape = (GAME_VEC_INPUT, ), dtype = "int32", name = "game_idx")
game_output = Dense(GAME_VEC_SIZE, activation = "softmax")(game_idx)

umpire_game = concatenate([umpire_embed, game_output], name = "umpire_game")

output_1 = Dense(6, activation = "relu")(umpire_game)
output = Dense(1, activation = ACTIVATION)(output_1)

model = Model(inputs = [umpire_idx, game_idx], outputs = [output])
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = "binary_crossentropy")

In [108]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
umpire_idx (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 16)        1728        umpire_idx[0][0]                 
__________________________________________________________________________________________________
umpire_embed (Reshape)          (None, 16)           0           embedding_1[0][0]                
__________________________________________________________________________________________________
game_idx (InputLayer)           [(None, 19)]         0                                            
____________________________________________________________________________________________

In [109]:
BATCH_SIZE = 100
NUM_EPOCHS = 100

X_list = [data["umpname"], data['game_condition']]
y = data["type"]
history = model.fit(X_list, y, epochs = NUM_EPOCHS, batch_size = BATCH_SIZE,
                    verbose = 2, shuffle = True)

Epoch 1/100
14292/14292 - 11s - loss: 0.0151
Epoch 2/100
14292/14292 - 12s - loss: 4.6017e-05
Epoch 3/100
14292/14292 - 14s - loss: 2.4954e-05
Epoch 4/100
14292/14292 - 13s - loss: 1.7012e-05
Epoch 5/100
14292/14292 - 13s - loss: 1.2863e-05
Epoch 6/100
14292/14292 - 13s - loss: 1.0324e-05
Epoch 7/100
14292/14292 - 14s - loss: 8.6182e-06
Epoch 8/100
14292/14292 - 13s - loss: 7.3890e-06
Epoch 9/100
14292/14292 - 14s - loss: 6.4795e-06
Epoch 10/100
14292/14292 - 13s - loss: 5.7584e-06
Epoch 11/100
14292/14292 - 13s - loss: 5.1831e-06
Epoch 12/100
14292/14292 - 13s - loss: 4.7213e-06
Epoch 13/100
14292/14292 - 14s - loss: 4.3339e-06
Epoch 14/100
14292/14292 - 14s - loss: 4.0049e-06
Epoch 15/100
14292/14292 - 15s - loss: 3.7262e-06
Epoch 16/100
14292/14292 - 15s - loss: 3.4861e-06
Epoch 17/100
14292/14292 - 13s - loss: 3.2772e-06
Epoch 18/100
14292/14292 - 17s - loss: 3.0960e-06
Epoch 19/100
14292/14292 - 14s - loss: 2.9338e-06
Epoch 20/100
14292/14292 - 16s - loss: 2.7844e-06
Epoch 21/100


In [111]:
len(model.layers)

9

In [112]:
e = model.layers[1]
umpire_vec = e.get_weights()[0]
print(umpire_vec.shape)
umpire_vec

(108, 16)


array([[ 0.02617189,  0.00475683, -0.04323339, ..., -0.02439371,
         0.02589765,  0.03744293],
       [-0.02232335, -0.0082554 ,  0.00302848, ...,  0.03452596,
         0.03559462,  0.03426271],
       [-0.0305091 , -0.02042859,  0.00942302, ..., -0.03133342,
        -0.02444358,  0.01949589],
       ...,
       [ 0.00944399, -0.02704297,  0.03349053, ..., -0.04066584,
        -0.01580216, -0.02074442],
       [-0.02812636, -0.04262241, -0.03478913, ..., -0.00370804,
         0.03576279,  0.04235822],
       [-0.03602662, -0.04952085,  0.01334326, ...,  0.02597615,
        -0.03993732, -0.04338853]], dtype=float32)

In [120]:
import io

# for tensorflow projector
out_uv = io.open('umpire_vecs.tsv', 'w', encoding='utf-8')
out_um = io.open('umpire_meta.tsv', 'w', encoding='utf-8')

for ump_num in range(NUM_UMPIRES):    
    umpire = cat_to_int_rev[ump_num]
    umpire_em = umpire_vec[ump_num]
    out_um.write(umpire + "\n")
    out_uv.write('\t'.join([str(x) for x in umpire_em]) + "\n")
        
    
out_uv.close()
out_um.close()

In [ ]:
# pitcher = pd.read_csv('pitcher_vecs_2011_.tsv', sep='\t', header = None)
# pitcher.columns = ['dim_'+str(i+1) for i in range(16)]
# pitcher_names = pd.read_csv('pitcher_meta_2011_.tsv', sep='\t', header = None)
# pitcher['Player'] = pitcher_names[0]
# pitcher